<a href="https://colab.research.google.com/github/louiezzang/next-gpt/blob/main/notebooks/llama2_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning Llama2
